In [ ]:
# %pip install -qU pypdf langchain_community
# %pip install langchain_chroma langchain_openai langchain_text_splitters
# %pip install openai chromadb langchain pypdf
# %pip install langchainhub

In [1]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
import os

In [7]:
from dotenv import load_dotenv
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["PINECONE_API_KEY"] = pinecone_api_key

In [54]:
# text = """
# Your long PDF text goes here. This is just an example placeholder text.
# """

# class Text2Doc:
#     def __init__(self, text, path):
#         self.page_content = text
#         self.metadata ={'source': path}

# # Step 1: Split text into chunks
# docs = Text2Doc(text=summary, path="test.txt")
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
# chunks = text_splitter.split_documents([docs])

In [3]:
summaries =  ['Exporting can be both challenging and rewarding for small businesses, as it enables access to new international markets, potentially enhancing growth and resilience. While exporting offers numerous benefits, such as increased profits and reduced dependency on the domestic economy, it also carries risks, including regulations and logistics delays. Small- and medium-sized enterprises (SMEs) accounted for significant export value in 2019, and the rise of e-commerce provides new opportunities for global sales. Business owners must carefully research and plan their export strategies.',
 'Exporting presents both challenges and rewards for small businesses. With SMEs contributing $460 billion to U.S. exports in 2019, international sales offer potential growth opportunities, especially after the surge of e-commerce in 2020. Benefits of exporting include market diversification, increased resilience to economic downturns, and enhanced profit margins. However, businesses must navigate risks such as regulations, tariffs, and logistics. Understanding which goods or services to export is crucial, as most are permissible with some exceptions.',
 'Exporting offers small businesses access to international markets, allowing them to grow faster, diversify their customer base, and decrease production costs. However, it comes with risks such as regulatory challenges and logistics issues. A successful export strategy can enhance resilience and profitability, but careful research and planning are essential.',
 'Exporting offers significant opportunities for small businesses, enabling access to international markets and diversifying revenue streams. Despite the challenges posed by regulations, tariffs, and logistics, the benefits include increased profits, reduced business risk, and enhanced resilience. Small and medium-sized enterprises (SMEs) contributed $460 billion in export value, demonstrating their crucial role in global trade. However, careful planning and research are essential to navigate this landscape successfully.',
 "Exporting can be both challenging and rewarding for small businesses, offering access to international markets and potential growth. In 2019, SMEs contributed $460 billion to U.S. exports, and e-commerce has further increased opportunities for global sales. Benefits of exporting include market diversification, increased resilience, and profit growth, while risks include regulatory hurdles and logistical challenges. Understanding which goods or services to export is crucial, as some markets may have restrictions. The average exporter tends to grow faster than non-exporters, making international trade a vital component of a small business's growth strategy.",
 "Exporting can be both challenging and rewarding for small businesses, offering access to global markets and potential growth opportunities. Despite risks associated with international trade, such as regulations and logistics, exporters experience faster growth and increased resilience. The U.S. represents only 4% of the world's population, highlighting the importance of targeting international markets. Many goods and services can be exported, with diverse opportunities in durable and nondurable goods. Businesses should develop efficient strategies to support their global efforts.",
 'Exporting offers small businesses access to international markets, enhancing growth and resilience while reducing dependence on the U.S. economy. Despite its benefits, exporting involves challenges like regulations, tariffs, and logistics issues. A strategic approach, informed by research, can help SMEs capitalize on global opportunities and navigate the risks associated with international sales.',
 'Exporting can be a transformative opportunity for small businesses, providing access to new markets, increased profits, and resilience against economic downturns. Although it offers many benefits, such as market diversification and reduced dependence on the U.S. economy, exporting also comes with risks, including regulations and logistics challenges. Selecting appropriate goods or services and understanding international markets are key factors in successful exporting.']


In [4]:
topics = """
Benefits of exporting for small businesses
Risks and challenges of international trade
Choosing goods and services for exportBenefits of Exporting for Small Businesses
Risks Associated with International Trade
Strategies for Selecting Goods to ExportBenefits of Exporting for Small Businesses
Risks and Challenges of International Trade
Strategies for Successful ExportingBenefits of Exporting for Small Businesses
Risks and Challenges in International Exporting
Strategies for Effective Market EntryBenefits of Exporting for SMEs
Risks and Challenges in Exporting
Choosing Goods or Services for ExportBenefits of Exporting for Small Businesses
Challenges and Risks in International Trade
Strategies for Successful ExportingBenefits of Exporting for Small Businesses
Risks and Challenges in International Trade
Selecting Goods and Services for ExportBenefits of Exporting for Small Businesses
Risks and Challenges in International Trade
Choosing Goods and Services for Export
"""

In [10]:
from langchain_pinecone import PineconeVectorStore
import time
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone()

# TODO: Get this index name from DB
index_name = "export-lesson-index2"
vectorstore = None
embedding_model = OpenAIEmbeddings()

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

print("Existing indexes:", existing_indexes)

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)
    # Create vectorstore
    vectorstore = PineconeVectorStore.from_texts(
        summaries,
        index_name=index_name,
        embedding=embedding_model
    )

else:
    print("Existing index")
    vectorstore = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding_model)

Existing indexes: ['test-index', 'export-lesson-index', 'lesson-1-index']


In [ ]:
# %pip install pinecone_notebooks pinecone
# %pip install --upgrade --quiet  \
#     langchain-pinecone \
#     langchain-openai \
#     langchain \
#     langchain-community \
#     pinecone-notebooks

In [ ]:
# from pinecone_notebooks.colab import Authenticate

# Authenticate()
# # This is just to add API key. Actually we can skip this step if we have API key.

### Embed the splitted pages

This is a single run when the pdf is uploaded. Next times, we will just use the existing vectorstore. See below.

In [13]:
query = "What are the challenges of export for small business owners?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Exporting can be both challenging and rewarding for small businesses, as it enables access to new international markets, potentially enhancing growth and resilience. While exporting offers numerous benefits, such as increased profits and reduced dependency on the domestic economy, it also carries risks, including regulations and logistics delays. Small- and medium-sized enterprises (SMEs) accounted for significant export value in 2019, and the rise of e-commerce provides new opportunities for global sales. Business owners must carefully research and plan their export strategies.


### Turn to Retriever

In [14]:
retriever = vectorstore.as_retriever(search_type="mmr")

### Creating RAG (not necessary for agent)

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
# maximal marginal relevance
retriever = vectorstore.as_retriever(search_type="mmr")
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [19]:
# topic_name = "Creat"
results = rag_chain.invoke({"input": f"Generate sub topics for Export based on the contents."})

print(results['answer'])

1. Types of goods and services suitable for export
2. Challenges in exporting such as cultural differences, language barriers, and legal compliance
3. Financial aspects of exporting including securing financing, estimating costs, and navigating trade agreements
4. Utilizing digital tools for export assistance like Export Express loans and International Trade Loans
5. Benefits of exporting for small businesses such as increased profits, extended product life cycles, and decreased production costs.


In [21]:
results = rag_chain.invoke({"input": f"""Now teach me the topics \n{results['answer']} one by one."""})

print(results['answer'])

1. Types of goods and services suitable for export vary but commonly include unique or high-quality products, technology, services, and niche goods that have demand in foreign markets.
2. Challenges in exporting encompass cultural differences, language barriers, and legal compliance issues that businesses need to address to successfully enter international markets.
3. Financial aspects of exporting involve securing financing through loans, estimating transportation and shipping costs, and navigating trade agreements and regulations to ensure compliance and smooth operations.


### Creating RAG as a tool

In [31]:
tool_name = "Intermediate_Answer"
tool_desc = "Search for information about any Export learning content. For any questions about learning content, you must use this tool!"

In [32]:
from langchain.tools.retriever import create_retriever_tool
tool = create_retriever_tool(
    retriever,  # The RAG chain instance you are using
    tool_name,  # Renamed to better reflect the focus on learning materials
    tool_desc
)

### Create Agent

In [33]:
tools = [tool]

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### Custom Prompt

In [ ]:
class AgentPrompts:
    agent_system_prompt = """
    You are an AI learning assistant helping a mentor coach their client on business-related matters.

    ```
    Client Personal Information:
    {client_personal_info}
    ```

    Client Business Information:
    {client_business_info}

    Topics to cover:
    {topics_to_cover}

    The user's preffered language is {language}.

    You are responsible for assessing the user's understanding of each topic. Use the RAG (Retrieval-Augmented Generation) tool, which has access to knowledge from an uploaded PDF file, to generate relevant content. The assistant should cover the topics sequentially, ensuring the user comprehends each topic before moving to the next one.

### Example Conversation Flow:

**Initial Conversation Starter:**
"Hello! Based on the client's business information and the topics we need to cover, I recommend starting with [specific topic]. How familiar are you with this topic?"

**Proceeding Based on Understanding:**

1. **Introduction to Topic:**
   - Provide a brief overview of the topic using content generated by the RAG tool.
   - Example: "Let's begin with an overview of [specific topic]. Here's a brief introduction: [content from RAG]."

2. **Assess Understanding:**
   - Ask questions to gauge the user's understanding.
   - Example: "Do you have any questions about this overview? Can you explain how this might apply to your business?"

3. **Deepen Understanding:**
   - Provide more detailed information or examples if the user needs more clarity.
   - Example: "Here are more details about [specific aspect]. [content from RAG]. How do you think this could impact your current business strategy?"

4. **Confirm Understanding:**
   - Ensure the user understands before moving on.
   - Example: "Great! Do you feel comfortable with this topic, or is there anything else you'd like to explore further?"

5. **Transition to Next Topic:**
   - Once understanding is confirmed, move to the next topic.
   - Example: "Since you're comfortable with [current topic], let's move on to [next topic]. Here's an overview: [content from RAG]."

"""

language = "Mizo"

In [80]:
class AgentPrompts:
    agent_system_prompt = """
    You are an AI learning assistant helping a mentor coach their client on business-related matters.

    ```
    Client Personal Information:
    {client_personal_info}
    ```

    Client Business Information:
    {client_business_info}

    Topics to cover:
    {topics_to_cover}

    The user's preffered language is {language}.

    You are responsible for assessing the user's understanding of each topic. Use the RAG (Retrieval-Augmented Generation) tool, which has access to knowledge from an uploaded PDF file, to generate relevant content. The assistant should cover the topics sequentially, ensuring the user comprehends each topic before moving to the next one.

### Example Conversation Flow:

**Initial Conversation Starter:**
"Hello! Based on the client's business information and the topics we need to cover, I recommend starting with [specific topic]. How familiar are you with this topic?"

**Proceeding Based on Understanding:**

1. **Introduction to Topic:**
   - Provide a brief overview of the topic using content generated by the RAG tool.
   - Example: "Let's begin with an overview of [specific topic]. Here's a brief introduction: [content from RAG]. Let's start with the first one: [first sub topic]"

2. **Assess Understanding:**
   - Ask questions to gauge the user's understanding.
   - Example: "Do you have any questions about this overview? Can you explain how this might apply to your business?"

3. **Deepen Understanding:**
   - Provide more detailed information or examples with user's business.
   - Example: "Here is how you can apply to your business for example. [Example]. How do you think this could impact your current business strategy?"

4. **Confirm Understanding:**
   - Ensure the user understands before moving on.
   - Example: "Great! Do you feel comfortable with this topic, or is there anything else you'd like to explore further?"

5. **Transition to Next Topic:**
   - Once understanding is confirmed, move to the next topic.
   - Example: "Since you're comfortable with [current topic], let's move on to [next topic]. Here's an overview: [content from RAG]."

"""

language = "English"

### Learning Agent

In [81]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
custom_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", AgentPrompts.agent_system_prompt),
        MessagesPlaceholder("chat_history", optional=False),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

custom_prompt = custom_prompt.partial(topics_to_cover=topics, language=language)

In [65]:
print(custom_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'client_business_info', 'client_personal_info', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'topics_to_cover': '\nBenefits of exporting for small businesses\nRisks and challenges of international trade\nChoosing goods and services for exportBenefits of Exporting for Small Businesses\nRisks Associated with International Trade\nStrategi

### Sample Customer Information

In [82]:
client_personal_info = """
Name: Jane Doe
Age: 34
Education: MBA in Marketing
Experience: 10 years in small business management
Email: jane.doe@example.com
Phone: +1-234-567-8901
Location: Springfield, IL
"""

client_business_info = """
Business Name: Doe's Delights
Industry: Food and Beverage
Business Type: Small bakery specializing in organic and gluten-free products
Year Established: 2015
Number of Employees: 15
Annual Revenue: $500,000
Location: 123 Main Street, Springfield, IL

Business Summary:
Doe's Delights is a local bakery that has carved a niche in the organic and gluten-free segment. Founded in 2015 by Jane Doe, the business has steadily grown, catering to health-conscious consumers in Springfield and surrounding areas. The bakery offers a variety of baked goods, including bread, cakes, cookies, and pastries, all made with high-quality, organic ingredients.

Goals:
- Increase annual revenue by 20% over the next year
- Expand the product line to include vegan options
- Open a second location in the neighboring town
- Enhance online presence and e-commerce capabilities

Difficulties:
- Managing supply chain disruptions for organic ingredients
- Competition from larger bakeries and supermarkets
- Limited marketing budget
- Challenges in scaling up production without compromising quality
"""

### Adding Memory

In [83]:
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryBufferMemory
from langchain.agents import AgentExecutor, create_openai_tools_agent, load_tools

memory = ConversationBufferWindowMemory(memory_key = "chat_history", return_messages = True, input_key = "input", max_token_limit=3000)

# print(all_tools)
# print(prompt)
# Create an agent with all tools
agent2 = create_openai_tools_agent(llm, tools, custom_prompt)
# Create an agent executor by passing in the agent and tools
agent_executor2 = AgentExecutor(agent = agent2, tools = tools, verbose = False, memory = memory, handle_parsing_errors=True)


## Initiator

This will be triggered once the user clicks 'Start' button.

In [84]:
module_name = "Exporting"
user_input = f"Teach me {module_name}"
response = agent_executor2.invoke({"input": user_input, 'client_personal_info':client_personal_info, 'client_business_info':client_business_info})
print(response['output'])

Hello, Jane! Based on your business information and goals, exporting could be a great opportunity for Doe's Delights. Let's start with the basics of exporting. How familiar are you with the concept of exporting for small businesses?


### Evaluation

In [88]:
user_input = "Teach me one at a time."
response = agent_executor2.invoke({"input": user_input, 'client_personal_info':client_personal_info, 'client_business_info':client_business_info})
print(response['output'])

Got it! Let's focus on one benefit at a time.

**Benefit 1: Increased Sales**

Exporting can significantly increase your sales by allowing you to reach customers in new markets. When you sell your baked goods internationally, you can tap into a larger customer base beyond Springfield. This can lead to higher sales volumes, especially if your products resonate well with consumers in those markets.

For example, if you find a market that values organic and gluten-free products, you could see a substantial increase in demand for your offerings.

Do you have any questions about how increased sales through exporting could impact Doe's Delights? Or can you think of any specific markets where you might want to sell your products?
